## Reconhecimento de digitais

In [1]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import os
import pickle

Funções para carregar e salvar informações das digitais

In [2]:
def writeFeatures(kp, des, fileName):
    index1 = []
    for point in kp:
        temp = (point.pt, point.size, point.angle, point.response, point.octave, 
            point.class_id) 
        index1.append(temp)
    file=open("keypoints/"+fileName+".pkl", "wb")
    pickle.dump(index1, file)
    file.close()
    file=open("descritores/"+fileName+".npy", "wb")
    np.save(file, des)
    file.close()   

def readFeatures(fileName):
    file=open("descritores/"+fileName+".npy", "rb")
    des=np.load(file)
    file.close()

    index = pickle.load(open("keypoints/"+fileName+".pkl", "rb"))
    kp = []

    for point in index:
        temp = cv2.KeyPoint(x=point[0][0],y=point[0][1],size=point[1], angle=point[2], 
                                response=point[3], octave=point[4], class_id=point[5]) 
        kp.append(temp)
    return [kp, des]

Funções para realizar pareamento entre features de digitais (a primeira função mostra uma imagem com o resultado visual da processo)

In [94]:
def SIFT_Match_Image(file1, file2):
    img1=cv2.imread(file1, cv2.IMREAD_GRAYSCALE)
    img2=cv2.imread(file2, cv2.IMREAD_GRAYSCALE)

    sift=cv2.SIFT_create(nOctaveLayers=4, contrastThreshold=0.12, sigma=2.5, enable_precise_upscale=True) 
    #valores sugeridos pelo artigo, com sigma exagerado pra remover pontos de interesse muito pequenos, gerados por regiões de baixa pressão
    kp1, des1=sift.detectAndCompute(img1, None)
    kp2, des2=sift.detectAndCompute(img2, None)

    index_params = dict(algorithm = 1, trees = 5) #alg=1 significa usar indexação em kdtree
    search_params = dict(checks=50)  #quantidade de pontos checados pelo knn
    
    flann = cv2.FlannBasedMatcher(index_params,search_params)
    
    matches = flann.knnMatch(des1,des2,k=2)
    
    # Need to draw only good matches, so create a mask
    matchesMask = [[0,0] for i in range(len(matches))]
    counter=0
    # ratio test as per Lowe's paper
    for i,(m,n) in enumerate(matches):
        if m.distance < 0.8*n.distance:
            counter+=1
            matchesMask[i]=[1,0]
    
    draw_params = dict(matchColor = (0,255,0),
                    singlePointColor = (255,0,0),
                    matchesMask = matchesMask,
                    flags = cv2.DrawMatchesFlags_DEFAULT)
    
    img3 = cv2.drawMatchesKnn(img1,kp1,img2,kp2,matches,None,**draw_params)
    
    plt.imshow(img3,),plt.show()
    
    return [counter, max(len(kp1),len(kp2))]

def SIFT_Match(kp1, des1, kp2, des2):
    if(len(kp1)<0.2*len(kp2) or 0.2*len(kp1)>len(kp2)):
        return [0, 1]
    index_params = dict(algorithm = 1, trees = 5) #alg=1 significa usar indexação em kdtree
    search_params = dict(checks=50)  #quantidade de pontos checados pelo knn
    
    flann = cv2.FlannBasedMatcher(index_params,search_params)
    
    matches = flann.knnMatch(des1,des2,k=2)
    
    counter=0
    # ratio test as per Lowe's paper
    for (m,n) in matches:
        if m.distance < 0.8*n.distance:
            counter+=1

    return [counter, max(len(kp1),len(kp2))]

Calcula as features de todas as digitais do banco de dados dado, são salvos no armazenamento secundário para futuro acesso

In [4]:
if not os.path.exists("keypoints"):
    os.makedirs("keypoints")

if not os.path.exists("descritores"):
    os.makedirs("descritores")

sift=cv2.SIFT_create(nOctaveLayers=4, contrastThreshold=0.12, sigma=2.5, enable_precise_upscale=True) 
for file in os.listdir("imagens"):
    img=cv2.imread("imagens/"+file, cv2.IMREAD_GRAYSCALE)
    kp, des=sift.detectAndCompute(img, None)
    file=file[0:len(file)-4]
    writeFeatures(kp, des, file)

Separa a base de dados em um conjunto de treino e de teste

In [ ]:
from random import randint

files=[]

for str in os.listdir("imagens"):
    files+=[str[:-4]]


dados=list()
for i in range(len(files)//8):
    dados+=[files[8*i:8*(i+1)]]

treino=list()
teste=list()
for digital in dados:
    indice=randint(0,7)
    teste+=["imagens/"+digital.pop(indice)+".tif"]
    treino+=[digital]

['imagens/012_3_4.tif', 'imagens/012_4_6.tif', 'imagens/012_5_8.tif', 'imagens/012_6_6.tif', 'imagens/012_7_8.tif', 'imagens/012_8_3.tif', 'imagens/013_3_8.tif', 'imagens/013_4_6.tif', 'imagens/013_5_6.tif', 'imagens/013_6_8.tif', 'imagens/013_7_4.tif', 'imagens/013_8_1.tif', 'imagens/017_3_4.tif', 'imagens/017_4_7.tif', 'imagens/017_5_5.tif', 'imagens/017_6_2.tif', 'imagens/017_7_4.tif', 'imagens/017_8_7.tif', 'imagens/022_3_4.tif', 'imagens/022_4_6.tif', 'imagens/022_5_4.tif', 'imagens/022_6_4.tif', 'imagens/022_7_2.tif', 'imagens/022_8_8.tif', 'imagens/027_3_7.tif', 'imagens/027_4_3.tif', 'imagens/027_5_2.tif', 'imagens/027_6_7.tif', 'imagens/027_7_5.tif', 'imagens/027_8_1.tif', 'imagens/045_3_6.tif', 'imagens/045_4_2.tif', 'imagens/045_5_1.tif', 'imagens/045_6_8.tif', 'imagens/045_7_8.tif', 'imagens/045_8_4.tif', 'imagens/047_3_3.tif', 'imagens/047_4_3.tif', 'imagens/047_5_6.tif', 'imagens/047_6_1.tif', 'imagens/047_7_6.tif', 'imagens/047_8_6.tif', 'imagens/057_3_3.tif', 'imagens/0

Realiza a classificação entre duas digitais, considera que qualquer valor acima de 8% de matches entre as features das digitais é um potencial emparelhamento entre as duas digitais. Faz isso para todas as digitais do conjunto de teste

In [ ]:
def getCod(str):
    index=str.find("/")+1
    return str[index:index+3]

def classificador(treino, imagem):
    imagem=cv2.imread(imagem, cv2.IMREAD_GRAYSCALE)
    sift=cv2.SIFT_create(nOctaveLayers=4, contrastThreshold=0.12, sigma=2.5, enable_precise_upscale=True) 
    kp2, des2=sift.detectAndCompute(imagem, None)
    matches=[]
    for digital in treino:
        score=0
        nivel=1
        for leitura in digital:
            args=readFeatures(leitura)
            res=SIFT_Match(args[0], args[1], kp2, des2)
            res=res[0]/res[1]
            if(res<0.02):
                if(nivel==0):
                    score=0
                    break
                nivel=0
            score+=res
        score=score/len(digital)
        if(score>0.08):
            matches.append([score, leitura[0:3]])
    return matches

acertos=0
falsoPositivo=0
for arq in teste:
    res=classificador(treino, arq)
    if res!=[]:
        if max(res, key=lambda x:x[0])[1]==getCod(arq):
            acertos+=1
        else:
            falsoPositivo+=1
print(acertos)
print(falsoPositivo)

hit
hit
hit
hit
hit
hit
hit
hit
hit
hit
hit
hit
hit
hit
hit
hit
hit
hit
hit
hit
hit
hit
hit
hit
hit
hit
hit
hit
hit
hit
hit
hit
hit
hit
hit
hit
hit
hit
hit
hit
hit
41
0


In [ ]:
teste=[]

for file in os.listdir("Falsas digitais"):
    teste+=["Falsas digitais/"+file]

acertos=0
falsoPositivo=0
for arq in teste:
    res=classificador(treino, arq)
    if res!=[]:
        falsoPositivo+=1
    else:
        acertos+=1

print(acertos)
print(falsoPositivo)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43


KeyboardInterrupt: 

Testes para digitais verdadeiras e falsas

In [ ]:
acertos=0
falsoPositivo=0
falsoNegativo=0

files=[]

for str in os.listdir("imagens"):
    files+=[str[:-4]]

falsas=[]
for file in os.listdir("Falsas digitais"):
    falsas+=["Falsas digitais/"+file]

for i in range(4): #repetir o experimento 4 vezes e comparar os resultados
    dados=[]
    for i in range(len(files)//8):
        dados+=[files[8*i:8*(i+1)]]
        treino=list()
        teste=list()
    for digital in dados:
        indice=randint(0,7)
        teste+=["imagens/"+digital.pop(indice)+".tif"]
        treino+=[digital]
    for arq in teste:
        res=classificador(treino, arq)
        if res!=[]:
            if max(res, key=lambda x:x[0])[1]==getCod(arq):
                acertos+=1
            else:
                falsoPositivo+=1
    
    for arq in falsas:
        res=classificador(treino, arq)
        if res!=[]:
            falsoPositivo+=1
        else:
            acertos+=1
    


TypeError: 'str' object is not callable

In [ ]:
total=51*4+80*4

print(total, acertos, total-acertos-falsoNegativo-falsoPositivo, falsoPositivo, falsoNegativo)


524 473 48 3 0
